In [ ]:
ops = {
        'reg_file': 'H:/DATA/M150329_MP009_2015-04-29_2_4_5_6_plane1.bin',
        'Ly':512,
        'Lx':512,
        'diameter':10,
        'tau':  1., # this is the only parameter used for deconvolution per se        
        'fs': 10.,  # sampling rate        
        'baseline': 'maximin', # baselining mode
        'win_baseline': 60., # window for maximin
        'sig_baseline': 10., # smoothing constant for gaussian filter 
        'prctile_baseline': 8.,# smoothing constant for gaussian filter        
        'neucoeff': .7,  # neuropil coefficient 
        'neumax': 1.,  # maximum neuropil coefficient (not implemented)
        'niterneu': 5, # number of iterations when the neuropil coefficient is estimated (not implemented)
        'maxregshift': 0.,
        'subpixel' : 10,
        'batch_size': 400, 
        'num_workers': 0, # 0 to select automatically, -1 to disable parallelism, N to enforce value
      }



In [ ]:
import register
imp.reload(register)

reg_file = open(ops['reg_file'], 'rb')

with open(ops['reg_file'], 'rb') as reg_file:
    buff = reg_file.read(Ly*Lx*nimgbatch*2)
    while buff != '':
        ## FIX LAST FRAME ISSUE !!!
        buff = reg_file.read(Ly*Lx*nimgbatch*2)
        data = np.frombuffer(buff, dtype=np.int16, offset=0)
        data = np.reshape(data.astype(np.float32), (Ly,Lx,nimgbatch), order='F')
        data = np.transpose(data, (2,0,1))
        # subtract mean of batch
        data = data - data.mean(axis=0)
        # bin data
        dbin = np.reshape(data, (nt0,-1,Ly,Lx))
        dbin = np.squeeze(dbin.mean(axis=0))
        print(dbin.shape)
        #plt.imshow(data[0,:,:])#.mean(axis=0))
        #plt.show()
        
        inds = ix + np.arange(0,dbin.shape[0])
        mov[inds,:,:] = dbin
        ix = ix + dbin.shape[0]

mov = np.reshape(mov, (mov.shape[0],Ly*Lx))